Importing relevant packages

In [1]:
#pip install -U epiweeks

In [2]:
import pandas as pd 
import numpy as np 
import pickle 
import datetime as dt
#import epiweeks as ew
from epiweeks import Week, Year
import os
import glob

Opening pickles!

In [3]:
with open('df_temp_rain.pickle','rb') as read_file:
    df_temp_rain = pickle.load(read_file)
with open('df_dengue.pickle','rb') as read_file:
    df_dengue = pickle.load(read_file)

Opening and merging search term CSVs

In [4]:
os.chdir("/Users/adelweiss/Documents/Project/Project 2/Data")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

counter = 0
df_st = pd.DataFrame
#combine all files in the list
for name in all_filenames:
    csvfile = pd.read_csv(name, skiprows=2)
    if counter == 0:
        df_st = csvfile 
        counter +=1
    else: df_st = df_st.merge(csvfile, how = 'left', on ='Week' )
    

In [5]:
df_st

,Week,fever: (Singapore),headache: (Singapore),ache + pain: (Singapore),dengue fever: (Singapore),eye pain: (Singapore),dengue: (Singapore),rashes: (Singapore)
0,2014-01-05,71,75,83,46,32,68,82
1,2014-01-12,84,52,77,29,57,61,73
2,2014-01-19,72,56,75,40,48,47,78
3,2014-01-26,66,69,73,18,43,34,84
4,2014-02-02,64,65,80,35,43,42,58
...,...,...,...,...,...,...,...,...
256,2018-12-02,82,70,77,24,59,29,54
257,2018-12-09,67,56,87,15,81,26,58
258,2018-12-16,77,76,84,27,61,28,60
259,2018-12-23,84,75,88,24,71,20,72


Dengue DataFrame 

In [6]:
df_dengue.info()
### There seems to be cases missing form Number column
### I think we can drop these after we merge the file together since Number is our Y 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 810 entries, 0 to 14
Data columns (total 4 columns):
Year            810 non-null int64
Eweek (Week)    810 non-null int64
Type dengue     810 non-null object
Number          796 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 31.6+ KB


In [7]:
###Keep only normal dengue cases 
df_dengue = df_dengue[df_dengue['Type dengue'] == 'Dengue']

In [8]:
df_dengue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 405 entries, 0 to 13
Data columns (total 4 columns):
Year            405 non-null int64
Eweek (Week)    405 non-null int64
Type dengue     405 non-null object
Number          398 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 15.8+ KB


Temperature & Rainfall

In [9]:
df_temp_rain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1827 entries, 0 to 30
Data columns (total 11 columns):
Date                             1827 non-null object
Daily Rainfall Total (mm)        1827 non-null float64
Highest 30-min Rainfall (mm)     1827 non-null object
Highest 60-min Rainfall (mm)     1827 non-null object
Highest 120-min Rainfall (mm)    1827 non-null object
Mean Temperature (°C)            1827 non-null float64
Maximum Temperature (°C)         1827 non-null float64
Minimum Temperature (°C)         1827 non-null float64
Mean Wind Speed (km/h)           1827 non-null object
Max Wind Speed (km/h)            1827 non-null object
Year                             1827 non-null int64
dtypes: float64(4), int64(1), object(6)
memory usage: 171.3+ KB


In [10]:
df_temp_rain.describe()

,Daily Rainfall Total (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Year
count,1827.000000,1827.000000,1827.000000,1827.000000,1827.000000
mean,4.631253,28.051013,31.654953,25.497865,2016.002189
std,10.299263,1.116300,1.471560,1.340702,1.414212
min,0.000000,23.000000,23.700000,21.100000,2014.000000
25%,0.000000,27.300000,30.900000,24.600000,2015.000000
50%,0.000000,28.100000,31.900000,25.500000,2016.000000
75%,3.600000,28.900000,32.600000,26.500000,2017.000000
max,75.400000,30.700000,35.000000,28.400000,2018.000000


In [11]:
### Need to turn the days into epi week
### Each epidemiological week begins on a Sunday and ends on Saturday

In [12]:
###Split date into day and month so that I can apply the epiweek conversion 
df_temp_rain['Date2'] = df_temp_rain['Date'].str.split() 
df_temp_rain['Day'] = df_temp_rain['Date2'].apply(lambda x:x[0])
df_temp_rain['Month'] = df_temp_rain['Date2'].apply(lambda x:x[1]) 

In [13]:
###apply dictionary to months so that we can use epi week to convert 
month_dictionary = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4,'May':5,
                    'Jun':6,'Jul':7,'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12} 

df_temp_rain['Month_Number']= df_temp_rain['Month'].apply(lambda x: month_dictionary[x])

In [14]:
### define function to apply convert epi weeks... 
def epi_week_conversion(row, df):
    data_date = dt.date(int(df['Year'].iloc[row]), int(df['Month_Number'].iloc[row]), int(df['Day'].iloc[row])) 
    return str(Week.fromdate(data_date))
#my_date = date(int(df.Year.iloc[row]), int(df.Month_Number.iloc[row]),int(df.Day.iloc)[row])int(df.Day.iloc)[row])))

In [15]:
### function needs to reference index..
## original index is out-of-order..so creating an index column by resetting index twice 
df_temp_rain.reset_index(inplace=True)
df_temp_rain.reset_index(inplace=True)

In [16]:
df_temp_rain['eweek'] = 1 #initialise column 
df_temp_rain['eweek'] = df_temp_rain['level_0'].apply(lambda x:epi_week_conversion(x,df_temp_rain))

In [17]:
df_temp_rain

,level_0,index,Date,Daily Rainfall Total (mm),Highest 30-min Rainfall (mm),Highest 60-min Rainfall (mm),Highest 120-min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h),Year,Date2,Day,Month,Month_Number,eweek
0,0,0,1 Dec,7.0,2.4,3.6,5.6,27.0,31.3,25.5,6.8,30.2,2018,"[1, Dec]",1,Dec,12,201848
1,1,1,2 Dec,0.4,0.2,0.2,0.4,27.5,31.2,25.4,8.6,31.3,2018,"[2, Dec]",2,Dec,12,201849
2,2,2,3 Dec,2.6,1.4,2,2.4,26.9,30.7,25.5,5,22,2018,"[3, Dec]",3,Dec,12,201849
3,3,3,4 Dec,0.0,0,0,0,27.8,31.9,24.9,7.6,26.3,2018,"[4, Dec]",4,Dec,12,201849
4,4,4,5 Dec,2.8,1.4,2.6,2.6,27.6,31.4,26.2,6.5,23.4,2018,"[5, Dec]",5,Dec,12,201849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822,1822,26,27 Jan,0.0,0,0,0,25.9,29.8,23.8,14.5,37.1,2014,"[27, Jan]",27,Jan,1,201405
1823,1823,27,28 Jan,0.0,0,0,0,26.5,30.1,24.4,15.4,39.6,2014,"[28, Jan]",28,Jan,1,201405
1824,1824,28,29 Jan,0.0,0,0,0,26.5,30.5,23.9,15.3,42.8,2014,"[29, Jan]",29,Jan,1,201405
1825,1825,29,30 Jan,0.0,0,0,0,25.8,30.6,22.4,12.2,38.9,2014,"[30, Jan]",30,Jan,1,201405


Search Terms DataFrame 

In [18]:
###Split date so that I can apply the epiweek conversion 
df_st['Week1'] = df_st['Week'].str.split('-') 
df_st['Day'] = df_st['Week1'].apply(lambda x:x[2])
df_st['Month_Number'] = df_st['Week1'].apply(lambda x:x[1]) 
df_st['Year'] = df_st['Week1'].apply(lambda x:x[0]) 

In [19]:
df_st.reset_index(inplace=True)

In [20]:
df_st['eweek'] = 1 #initialise column 
df_st['eweek'] = df_st['index'].apply(lambda x: epi_week_conversion(x,df_st))

In [21]:
df_st

,index,Week,fever: (Singapore),headache: (Singapore),ache + pain: (Singapore),dengue fever: (Singapore),eye pain: (Singapore),dengue: (Singapore),rashes: (Singapore),Week1,Day,Month_Number,Year,eweek
0,0,2014-01-05,71,75,83,46,32,68,82,"[2014, 01, 05]",05,01,2014,201402
1,1,2014-01-12,84,52,77,29,57,61,73,"[2014, 01, 12]",12,01,2014,201403
2,2,2014-01-19,72,56,75,40,48,47,78,"[2014, 01, 19]",19,01,2014,201404
3,3,2014-01-26,66,69,73,18,43,34,84,"[2014, 01, 26]",26,01,2014,201405
4,4,2014-02-02,64,65,80,35,43,42,58,"[2014, 02, 02]",02,02,2014,201406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,256,2018-12-02,82,70,77,24,59,29,54,"[2018, 12, 02]",02,12,2018,201849
257,257,2018-12-09,67,56,87,15,81,26,58,"[2018, 12, 09]",09,12,2018,201850
258,258,2018-12-16,77,76,84,27,61,28,60,"[2018, 12, 16]",16,12,2018,201851
259,259,2018-12-23,84,75,88,24,71,20,72,"[2018, 12, 23]",23,12,2018,201852
